<a href="https://colab.research.google.com/github/Guilhermesena-dev/Modelo_regrss-o_telgramdf/blob/main/TB05_MINERA%C3%87%C3%83O_DE_DADOS_GUILHERME_CAMPELO_SANTIAGO_DE_SENA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Link para video no youtube: https://youtu.be/EtwEto0ZXns

Ler o dataset fakeTelegram.BR_2022.csv

In [7]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import KFold, cross_val_score
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/fakeTelegram.BR_2022.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,5,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Texto,telegram,NaN,NaN
1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",5,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Texto,telegram,NaN,NaN
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,"É isso, nossa parte já foi quase toda feita. N...",5,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Texto,telegram,NaN,NaN
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,5,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,NaN,True,False,False,NaN,5,2022-10-05 06:28:29.316325,NaN,NaN,507185,Imagem,telegram,NaN,5ad5c8bbe9da93a37fecf3e5aa5b0637


Remova os trava-zaps, as linhas repetidas (duplicadas) e textos com menos de 5
palavras.

In [8]:
df_limpo = df[df["trava_zap"] == False].copy() #Remoção dos trava_zaps


In [9]:
df_limpo.drop_duplicates(inplace=True) #remoção de linhas duplicadas

In [10]:
def contar_palavras(texto): #remoção de textos com menos de 5 caracteres
    if pd.isna(texto):
        return 0
    return len(texto.strip().split())

df_limpo["n_palavras"] = df_limpo["text_content_anonymous"].apply(contar_palavras)
df_filtrado = df_limpo[df_limpo["n_palavras"] >= 5].copy()

df_filtrado.head(), len(df_filtrado)

(          date_message               id_member_anonymous  \
 0  2022-10-05 06:25:04  1078cc958f0febe28f4d03207660715f   
 1  2022-10-05 06:25:08                               NaN   
 2  2022-10-05 06:26:28  92a2d8fd7144074f659d1d29dc3751da   
 3  2022-10-05 06:27:28  d60aa38f62b4977426b70944af4aff72   
 5  2022-10-05 06:28:30                               NaN   
 
                  id_group_anonymous                                 media  \
 0  12283e08a2eb5789201e105b34489ee7                                   NaN   
 1  12283e08a2eb5789201e105b34489ee7                                   NaN   
 2  9f2d7394334eb224c061c9740b5748fc                                   NaN   
 3  c8f2de56550ed0bf85249608b7ead93d  94dca4cda503100ebfda7ce2bcc060eb.jpg   
 5  b52442a5fbc459ae590dca0d215e32f9                                   NaN   
 
   media_type media_url  has_media  has_media_url  trava_zap  \
 0        NaN       NaN      False          False      False   
 1        NaN       NaN      False

Agrupe as linhas com postagens iguais ou extremamente semelhantes. Aqui você
pode utilizar uma métrica de semelhança de textos. Crie uma variável para
representar a quantidade de vezes que a mensagem foi compartilhada. Observe que
ao agrupar linhas que possuem a “mesma” postagem (texto), você deve escolher
como valor para as variáveis data e hora da postagem, os valores da cópia mais
antiga

In [6]:
#eu não consegui processar todos os dados, pois todas as minhas tentativas a memoria ram do notebook ou do meu computador foi completamente utilizada e o codigo quebrou antes do fim do processamento

df = df.sample(n=30000, random_state=42).reset_index(drop=True)
df['text_content_anonymous'] = df['text_content_anonymous'].astype(str)


# Vetorização
vectorizer = TfidfVectorizer(stop_words=None)
tfidf = vectorizer.fit_transform(df["text_content_anonymous"])

# Agrupamento por similaridade
nn = NearestNeighbors(radius=0.05, metric='cosine', n_jobs=-1)
nn.fit(tfidf)
vizinhos = nn.radius_neighbors(tfidf, return_distance=False)

agrupados = set()
mensagens = []

for i, grupo_indices in enumerate(vizinhos):
    if i in agrupados:
        continue
    grupo = [idx for idx in grupo_indices if idx not in agrupados]
    if not grupo:
        continue
    for idx in grupo:
        agrupados.add(idx)
    base = df.iloc[grupo].sort_values("date_message").iloc[0].copy()
    base["compartilhamentos"] = len(grupo)
    mensagens.append(base)

df_final = pd.DataFrame(mensagens)

d) Você pode criar novos atributos numéricos, tais como: quantidade de palavras,
quantidade de caracteres etc.


In [11]:
df_final["qtd_palavras"] = df_final["text_content_anonymous"].apply(lambda x: len(str(x).split()))
df_final["qtd_caracteres"] = df_final["text_content_anonymous"].apply(lambda x: len(str(x)))


e) O modelo de regressão: regressão linear multivariada, regressão polinomial
multivariada;


In [12]:
model_linear = LinearRegression()
model_poly = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())


X = df_final[["qtd_palavras", "qtd_caracteres"]]
y = df_final["compartilhamentos"]

kf = KFold(n_splits=5, shuffle=True, random_state=42)
mse_linear = -cross_val_score(model_linear, X, y, scoring="neg_mean_squared_error", cv=kf).mean()
mse_poly = -cross_val_score(model_poly, X, y, scoring="neg_mean_squared_error", cv=kf).mean()

print("MSE Regressão Linear:", mse_linear)
print("MSE Regressão Polinomial:", mse_poly)


MSE Regressão Linear: 1969.034176502321
MSE Regressão Polinomial: 1969.094782049313


g) Regularização: Com regularização (Ridge, Lasso ou ElasticNet) e sem
regularização;


In [13]:
X = df_final[["qtd_palavras", "qtd_caracteres"]]
y = df_final["compartilhamentos"]

kf = KFold(n_splits=5, shuffle=True, random_state=42)
mse_ridge = -cross_val_score(Ridge(), X, y, scoring="neg_mean_squared_error", cv=kf).mean()
mse_lasso = -cross_val_score(Lasso(), X, y, scoring="neg_mean_squared_error", cv=kf).mean()
mse_elastic = -cross_val_score(ElasticNet(), X, y, scoring="neg_mean_squared_error", cv=kf).mean()


h) Normalização dos dados: sem normalização, Z-Score, Min-Max (OPCIONAL);


In [14]:
scaler_z = StandardScaler()
X_z = scaler_z.fit_transform(X)
mse_z = -cross_val_score(Ridge(), X_z, y, scoring="neg_mean_squared_error", cv=kf).mean()

scaler_mm = MinMaxScaler()
X_mm = scaler_mm.fit_transform(X)
mse_mm = -cross_val_score(Ridge(), X_mm, y, scoring="neg_mean_squared_error", cv=kf).mean()

l) Treinamento, Validação e Teste: Outer K-Fold Cross-Validation;

In [16]:
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1, 2), max_features=1000)),  # bigramas
    ("scaler", StandardScaler(with_mean=False)),
    ("regressor", Ridge())
])

textos = df_final["text_content_anonymous"]
mse_textual = -cross_val_score(pipeline, textos, y, scoring="neg_mean_squared_error", cv=kf).mean()

print("MSE Textual:", mse_textual)

MSE Textual: 1974.348620512928
